In [26]:
import glob
import pandas as pd

In [27]:
list = glob.glob('*.xlsx')

In [28]:
main_df = pd.DataFrame()
name_row_v6 = ('id_pay', 'data', 'sum', 'phone', 'type', 'name')
name_row_v7 = ('id_pay', 'data', 'sum', 'phone', 'type', 'name', 'period')
name_row_v8 = ('id_pay', 'data', 'sum', 'phone',
               'type', 'name', 'member_card', 'period')


In [29]:
uncorrect_data = []

for index,path in enumerate(list):
    try:
        df = pd.read_excel(path)
        if len(df.columns) == 6:
            df.set_axis(name_row_v6, axis=1, inplace=True)
        elif len(df.columns) == 7:
            df.set_axis(name_row_v7, axis=1, inplace=True)
        else:
            df.set_axis(name_row_v8, axis=1, inplace=True)    
        main_df = pd.concat([main_df, df], )
    except:
        print(path)
        uncorrect_data.append(path)
        
print(len(uncorrect_data))
        

duplicat.xlsx
1


In [30]:
main_df.head(15)
main_df = main_df.dropna(subset=['sum'])

In [31]:
main_df = main_df[main_df['id_pay'] != 'ID платежа']

In [32]:
main_df = main_df.reset_index(drop=True)

In [33]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3192 entries, 0 to 3191
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id_pay       3192 non-null   object
 1   data         3192 non-null   object
 2   sum          3192 non-null   object
 3   phone        3192 non-null   object
 4   type         3192 non-null   object
 5   name         3192 non-null   object
 6   period       77 non-null     object
 7   member_card  77 non-null     object
dtypes: object(8)
memory usage: 199.6+ KB


In [10]:
import re

In [149]:
def phones_update(row):
    '''change uncorrect phone to correct format

    Args:
        row (str): uncorrect number
    return:
        phone (int): correct format of number
    '''
    regex_num = re.compile('\d+')
    phone = regex_num.findall(row)
    phone = ''.join(phone)
    if len(phone) == 10 and phone[0] == '9':
        phone = '8' + phone
    if phone[0] == '7' and len(phone) == 11:
        phone = phone.replace('7','8',1)
        return phone
    if len(phone) != 11 or phone[0] != '8':
        return '0'
    return phone

In [34]:
main_df.head(5)

,id_pay,data,sum,phone,type,name,period,member_card
0,84047719,2021-02-28 00:42:54.460000,1300,89161329425,1. Единоразовый вступительный взнос,Овчинникова Александра Андреевна,NaN,NaN
1,84047877,2021-02-28 00:48:41.720000,300,89161329425,2. Ежегодный членский взнос,Овчинникова Александра Андреевна,NaN,NaN
2,84101338,2021-02-28 11:38:59.370000,600,89778129510,2. Ежегодный членский взнос,Пахомова Екатерина Николаевна,NaN,NaN
3,84193569,2021-02-28 22:29:56.430000,1300,+79039663778,1. Единоразовый вступительный взнос,Козлов Владимир Николаевич,NaN,NaN
4,84194252,2021-02-28 22:39:40.190000,1300,+79295059935,1. Единоразовый вступительный взнос,Иванов Алексей Владимирович,NaN,NaN


In [35]:
# main_df['phone'] = pd.to_numeric(main_df['phone'], errors='coerce')
main_df['sum'] = pd.to_numeric(main_df['sum'], errors='coerce')
main_df['new_data'] = pd.to_datetime(main_df['data'], errors='coerce', format='%Y-%m-%d')

In [36]:
from pandas import ExcelWriter
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3192 entries, 0 to 3191
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id_pay       3192 non-null   object        
 1   data         3192 non-null   object        
 2   sum          3192 non-null   float64       
 3   phone        3192 non-null   object        
 4   type         3192 non-null   object        
 5   name         3192 non-null   object        
 6   period       77 non-null     object        
 7   member_card  77 non-null     object        
 8   new_data     3192 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 224.6+ KB


In [37]:
main_df = main_df.drop_duplicates(subset=['id_pay'])

In [38]:
main_df['sum'].sum()

1178407.38

In [23]:
dupl = main_df[main_df.duplicated(subset='name',keep=False)]

In [25]:
main_df['sum'].describe()

count     1624.000000
mean       707.824741
std        539.736317
min        300.000000
25%        300.000000
50%        300.000000
75%       1300.000000
max      10000.000000
Name: sum, dtype: float64

In [39]:
main_df = main_df.reset_index(drop=True)

In [24]:
with ExcelWriter("duplicat.xlsx") as writer:
    dupl.to_excel(writer, sheet_name='sheet1')

In [40]:
with ExcelWriter("main_equaring.xlsx") as writer:
    main_df.to_excel(writer, sheet_name="Sheet1")